# School Location Choice Model

In this notebook, a simple logit location choice model is used to predict university campus choice. Data from the 2015 StudentMoveTO data is used. The model is run separately on each of the 13 segments we have previously outlined.

First, we install Biogeme and Pandas, and load the original data.

In [1]:
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.models as models
from biogeme.expressions import Beta, DefineVariable
import biogeme.messaging as msg
import pandas as pd

hh_df = pd.read_csv('../Data/SMTO_2015/SMTO_2015_Households.csv')
ps_df = pd.read_csv('../Data/SMTO_2015/SMTO_2015_Respondents.csv')

Now, we prepare a dataframe which will be used as input for Biogeme by selecting and manipulating the relevant columns from the data. We use a function to convert information about Level, Status, Years of Involvement, and Living Arrangement into a column containing the respondent's Segment.

In [2]:
def info_to_segment(x):
    if x.Level == 'Other':
        return 0
    elif x.Level == 'UG':
        return (0 if x.Status == 'FT' else 2) + (not x.Family) + (1 if x.Years == 0 else 5)
    else:
        return 9 + (0 if x.Status == 'FT' else 2) + (not x.Family)

# Load relevant columns
df = ps_df[['pscampusattend', 'personstatusgrad', 'personstatustime', 'psuniversityinvolvednumyears']]
df = df.join(hh_df[['HmTTS2006', 'hhlivingsituation']])
df = df.rename(columns={'HmTTS2006': 'HomeZone', 'pscampusattend': 'Campus', 'hhlivingsituation': 'Family', 
                       'personstatusgrad': 'Level', 'personstatustime': 'Status', 'psuniversityinvolvednumyears': 'Years'})
df = df.dropna() # Remove rows with missing data

# Convert Campus column to numerical column
campus_name_to_num = {"Downtown Toronto (St. George)": 0, "Scarborough (UTSC)": 1, "Mississauga (UTM)": 2,
                      "Keele": 3, "Glendon": 4, "RyersonU": 5, "OCADu": 6}
df.replace({'Campus': campus_name_to_num}, inplace=True)
df['HomeZone'] = pd.to_numeric(df['HomeZone'], downcast='signed')
df['Family'] = (df['Family'] == 'Live with family/parents')*1
df['Segment'] = df.apply(info_to_segment, axis=1)
df['Available'] = 1 # All campuses available to all students

df['Segment'].value_counts()

5     4914
6     2438
10    2271
1     2012
2     1024
9      872
0      318
7      315
8      279
12     243
11     109
3       69
4       46
Name: Segment, dtype: int64

Keeping only the relevant columns, we have:

In [3]:
df = df[['Campus', 'HomeZone', 'Segment', 'Available']]
df

,Campus,HomeZone,Segment,Available
0,1,261,5,1
1,0,71,10,1
2,0,3714,5,1
3,0,74,6,1
4,0,71,10,1
...,...,...,...,...
15221,3,212,2,1
15222,3,233,5,1
15223,3,95,6,1
15224,3,2221,5,1


Now we must load information about the distance between each respondent's home zone and each campus. First, we load the data from the LoS matrix.

In [4]:
# Dataframe with walk distances
df_path = pd.read_csv('../../LoS/Walk_Distances.csv')
origins = list(set(list(df_path['Origin'])))
dists = list(df_path['Data'])

Now, we declare a function to look up the walk distance given an origin-destination pair.

In [5]:
not_found = set()
# Function for distance lookup
def find_distance(origin, destination):
    try:
        i = origins.index(origin)
    except ValueError:
        not_found.add(origin)
        return 0
    try:
        j = origins.index(destination)
    except ValueError:
        not_found.add(destination)
        return 0
    return dists[i*2392 + j] / 1000

We run this function for each row on each school zone, creating seven new columns.

In [6]:
# List of campus' TTS zones from Joven's MOE data
campus_zones = [69, 566, 3631, 391, 225, 38, 67]

# Load distances in dataframe
for i in range(7):
    df["Dist" + str(i)] = df['HomeZone'].apply(lambda x: find_distance(x, campus_zones[i]))
len(not_found)

127

127 zones were not found as they are outside the GTHA. Let's see what our dataframe looks like now:

In [7]:
df

,Campus,HomeZone,Segment,Available,Dist0,Dist1,Dist2,Dist3,Dist4,Dist5,Dist6
0,1,261,5,1,10.256060,14.88098,29.20657,22.59214,9.218413,9.580635,11.241730
1,0,71,10,1,1.132351,23.03920,19.64290,15.87906,11.211150,2.675173,2.723838
2,0,3714,5,1,23.319230,45.63271,4.51742,28.58045,32.555200,24.964000,23.686150
3,0,74,6,1,0.699414,24.11954,19.43932,16.81186,12.830410,2.314008,1.541276
4,0,71,10,1,1.132351,23.03920,19.64290,15.87906,11.211150,2.675173,2.723838
...,...,...,...,...,...,...,...,...,...,...,...
15221,3,212,2,1,7.120260,19.01731,23.76046,14.19530,5.732956,6.553092,8.193741
15222,3,233,5,1,15.917590,12.03644,32.96591,17.68772,6.019180,15.242170,16.903260
15223,3,95,6,1,2.783940,25.09743,17.84462,15.53600,12.829600,4.733398,3.979057
15224,3,2221,5,1,23.379880,26.15476,37.35434,13.30458,15.379040,23.250580,24.518920


We are now ready to run our model! We do this 13 times, each time keeping only one segment.

In [8]:
for i in range(13):
    temp_df = df.copy()
    database = db.Database("SMTO", temp_df)
    globals().update(database.variables)
    database.remove(Dist0 == 0) # Remove unknown distances
    
    database.remove(Segment != i)
    
    # Beta initialization: (name, value, lowerbound, upperbound, status, desc='')
    # Status 0 if estimated, 1 if maintained - reference choice should be 1
    ASC_SG = Beta('ASC_SG', 0, None, None, 0)
    ASC_SC = Beta('ASC_SC', 0, None, None, 0)
    ASC_MI = Beta('ASC_MI', 0, None, None, 0)
    ASC_YK = Beta('ASC_YK', 0, None, None, 0)
    ASC_YG = Beta('ASC_YG', 0, None, None, 1)
    ASC_RY = Beta('ASC_RY', 0, None, None, 0)
    ASC_OC = Beta('ASC_OC', 0, None, None, 0)
    B_DIST = Beta('B_DIST', 0, None, None, 0)

    # Variables: from columns in database
    AV = DefineVariable('AV', Available, database)
    SG_DIST = DefineVariable('SG_DIST', Dist0, database)
    SC_DIST = DefineVariable('SC_DIST', Dist1, database)
    MI_DIST = DefineVariable('MI_DIST', Dist2, database)
    YK_DIST = DefineVariable('YK_DIST', Dist3, database)
    YG_DIST = DefineVariable('YG_DIST', Dist4, database)
    RY_DIST = DefineVariable('RY_DIST', Dist5, database)
    OC_DIST = DefineVariable('OC_DIST', Dist6, database)

    # Utility Functions: note ASC_SG is 0
    V0 = ASC_SG + B_DIST * SG_DIST
    V1 = ASC_SC + B_DIST * SC_DIST
    V2 = ASC_MI + B_DIST * MI_DIST
    V3 = ASC_YK + B_DIST * YK_DIST
    V4 = ASC_YG + B_DIST * YG_DIST
    V5 = ASC_RY + B_DIST * RY_DIST
    V6 = ASC_OC + B_DIST * OC_DIST

    V  = {0: V0, 1: V1, 2: V2, 3: V3, 4: V4, 5: V5, 6: V6}
    av = {0: AV, 1: AV, 2: AV, 3: AV, 4: AV, 5: AV, 6: AV}
    
    logprob = models.loglogit(V, av, Campus)
    logger = msg.bioMessage()
    logger.setDebug()
    biogeme = bio.BIOGEME(database, logprob, numberOfThreads=1)
    biogeme.modelName = "SMTO_Segmented_Output/SMTO_Campus_Choice_Segment_" + str(i)
    results = biogeme.estimate(saveIterations=True)
    biogeme.createLogFile()
    betas = results.getBetaValues()
    for k,v in betas.items():
        print(f"{k:10}=\t{v:.3g}")
    pandasResults = results.getEstimatedParameters()
    print(pandasResults)

[10:32:23] < General >   Remove 10 unused variables from the database as only 9 are used.
[10:32:23] < General >   Log likelihood (N=309):  -601.2862
[10:32:23] < General >   Minimize with tol 1e-07
[10:32:23] < Debug >     ASC_MI:          0
[10:32:23] < Debug >     ASC_OC:          0
[10:32:23] < Debug >     ASC_RY:          0
[10:32:23] < Debug >     ASC_SC:          0
[10:32:23] < Debug >     ASC_SG:          0
[10:32:23] < Debug >     ASC_YK:          0
[10:32:23] < Debug >     B_DIST:          0
[10:32:23] < General >   Log likelihood (N=309):  -601.2862 Gradient norm:      2e+03  
[10:32:23] < Debug >     ASC_MI: -0.02249692
[10:32:23] < Debug >     ASC_OC: -0.02623158
[10:32:23] < Debug >     ASC_RY: 0.07771662
[10:32:23] < Debug >     ASC_SC: -0.02311936
[10:32:23] < Debug >     ASC_SG: 0.01858441
[10:32:23] < Debug >     ASC_YK: -0.003823587
[10:32:23] < Debug >     B_DIST: -0.9959274
[10:32:23] < General >   Log likelihood (N=309):  -1887.277 Gradient norm:      2e+03  
[10:

[10:32:23] < Debug >     ASC_MI: -0.3160336
[10:32:23] < Debug >     ASC_OC:   -2.03486
[10:32:23] < Debug >     ASC_RY:   2.548025
[10:32:23] < Debug >     ASC_SC: -0.4397623
[10:32:23] < Debug >     ASC_SG:   1.697767
[10:32:23] < Debug >     ASC_YK:   1.193664
[10:32:23] < Debug >     B_DIST: -0.09468609
[10:32:23] < General >   Log likelihood (N=309):    -357.81 Gradient norm:          3  
[10:32:23] < Debug >     ASC_MI: -0.3154496
[10:32:23] < Debug >     ASC_OC:   -2.03704
[10:32:23] < Debug >     ASC_RY:   2.550598
[10:32:23] < Debug >     ASC_SC: -0.4406585
[10:32:23] < Debug >     ASC_SG:    1.69716
[10:32:23] < Debug >     ASC_YK:   1.193871
[10:32:23] < Debug >     B_DIST: -0.09499147
[10:32:23] < General >   Log likelihood (N=309):  -357.8094 Gradient norm:          1  
[10:32:23] < Debug >     ASC_MI: -0.3156752
[10:32:23] < Debug >     ASC_OC:  -2.034841
[10:32:23] < Debug >     ASC_RY:   2.550071
[10:32:23] < Debug >     ASC_SC: -0.4399891
[10:32:23] < Debug >     ASC_S

[10:32:24] < General >   Results saved in file SMTO_Segmented_Output/SMTO_Campus_Choice_Segment_0.html
[10:32:24] < General >   Results saved in file SMTO_Segmented_Output/SMTO_Campus_Choice_Segment_0.pickle
[10:32:24] < General >   Log file created: SMTO_Segmented_Output/SMTO_Campus_Choice_Segment_0.log
ASC_MI    =	-0.255
ASC_OC    =	-1.8
ASC_RY    =	2.63
ASC_SC    =	-0.289
ASC_SG    =	1.77
ASC_YK    =	1.27
B_DIST    =	-0.0949
           Value   Std err    t-test       p-value  Rob. Std err  Rob. t-test  \
ASC_MI -0.255495  0.487220 -0.524394  6.000047e-01      0.459624    -0.555878   
ASC_OC -1.799090  0.770065 -2.336283  1.947648e-02      0.774038    -2.324292   
ASC_RY  2.629828  0.313975  8.375919  0.000000e+00      0.318148     8.266044   
ASC_SC -0.288939  0.500975 -0.576754  5.641055e-01      0.516059    -0.559896   
ASC_SG  1.774762  0.325967  5.444611  5.191874e-08      0.328226     5.407133   
ASC_YK  1.267303  0.347069  3.651449  2.607649e-04      0.344103     3.682923   
B

[10:32:25] < Debug >     ASC_MI:  0.9266359
[10:32:25] < Debug >     ASC_OC: -0.2498007
[10:32:25] < Debug >     ASC_RY:   1.883655
[10:32:25] < Debug >     ASC_SC:  0.9516079
[10:32:25] < Debug >     ASC_SG:   1.971169
[10:32:25] < Debug >     ASC_YK:   1.895051
[10:32:25] < Debug >     B_DIST: -0.06579851
[10:32:25] < General >   Log likelihood (N=1984):  -3181.982 Gradient norm:      9e+01  
[10:32:25] < Debug >     ASC_MI:   1.046445
[10:32:25] < Debug >     ASC_OC: 0.09268343
[10:32:25] < Debug >     ASC_RY:   2.023957
[10:32:25] < Debug >     ASC_SC:    1.04279
[10:32:25] < Debug >     ASC_SG:   2.048052
[10:32:25] < Debug >     ASC_YK:   1.972311
[10:32:25] < Debug >     B_DIST: -0.06574397
[10:32:25] < General >   Log likelihood (N=1984):  -3177.143 Gradient norm:      4e+01  
[10:32:25] < Debug >     ASC_MI:   1.268982
[10:32:25] < Debug >     ASC_OC:  0.2116194
[10:32:25] < Debug >     ASC_RY:   2.240821
[10:32:25] < Debug >     ASC_SC:   1.279487
[10:32:25] < Debug >     ASC

[10:32:26] < General >   Results saved in file SMTO_Segmented_Output/SMTO_Campus_Choice_Segment_1.html
[10:32:26] < General >   Results saved in file SMTO_Segmented_Output/SMTO_Campus_Choice_Segment_1.pickle
[10:32:26] < General >   Log file created: SMTO_Segmented_Output/SMTO_Campus_Choice_Segment_1.log
ASC_MI    =	0.976
ASC_OC    =	0.203
ASC_RY    =	2.04
ASC_SC    =	1.09
ASC_SG    =	1.99
ASC_YK    =	2.01
B_DIST    =	-0.0657
           Value   Std err     t-test       p-value  Rob. Std err  \
ASC_MI  0.975690  0.145308   6.714610  1.885714e-11      0.142198   
ASC_OC  0.202513  0.167988   1.205523  2.280015e-01      0.169032   
ASC_RY  2.041118  0.129606  15.748657  0.000000e+00      0.130574   
ASC_SC  1.094376  0.142888   7.658997  1.865175e-14      0.142966   
ASC_SG  1.992882  0.129779  15.355975  0.000000e+00      0.130782   
ASC_YK  2.014415  0.128623  15.661412  0.000000e+00      0.128758   
B_DIST -0.065668  0.003229 -20.339343  0.000000e+00      0.003009   

        Rob. t-te

...

[10:32:52] < General >   Results saved in file SMTO_Segmented_Output/SMTO_Campus_Choice_Segment_11.html
[10:32:52] < General >   Results saved in file SMTO_Segmented_Output/SMTO_Campus_Choice_Segment_11.pickle
[10:32:52] < General >   Log file created: SMTO_Segmented_Output/SMTO_Campus_Choice_Segment_11.log
ASC_MI    =	23.3
ASC_OC    =	24.2
ASC_RY    =	25.9
ASC_SC    =	-22.9
ASC_SG    =	27.5
ASC_YK    =	27
B_DIST    =	-0.095
            Value        Std err         t-test   p-value   Rob. Std err  \
ASC_MI  23.289563   1.141991e+05   2.039382e-04  0.999837   1.604277e+03   
ASC_OC  24.216601   1.141991e+05   2.120559e-04  0.999831   1.661404e+03   
ASC_RY  25.898560   1.141991e+05   2.267842e-04  0.999819   1.553068e+03   
ASC_SC -22.920661  1.797693e+308 -1.275004e-307  1.000000  1.797693e+308   
ASC_SG  27.495011   1.141991e+05   2.407637e-04  0.999808   1.663274e+03   
ASC_YK  26.972314   1.141991e+05   2.361867e-04  0.999812   1.667009e+03   
B_DIST  -0.094952   2.550929e-02  -3.72

[10:32:54] < Debug >     ASC_RY:  0.3761717
[10:32:54] < Debug >     ASC_SC:  -1.110123
[10:32:54] < Debug >     ASC_SG:   2.708955
[10:32:54] < Debug >     ASC_YK:   2.036386
[10:32:54] < Debug >     B_DIST: -0.07135081
[10:32:54] < General >   Log likelihood (N=233):  -246.0783 Gradient norm:      2e+01  
[10:32:54] < Debug >     ASC_MI:  -1.591431
[10:32:54] < Debug >     ASC_OC: -0.4640098
[10:32:54] < Debug >     ASC_RY:   0.647685
[10:32:54] < Debug >     ASC_SC:  -1.147971
[10:32:54] < Debug >     ASC_SG:   2.632859
[10:32:54] < Debug >     ASC_YK:   2.015737
[10:32:54] < Debug >     B_DIST: -0.06758366
[10:32:54] < General >   Log likelihood (N=233):  -243.7354 Gradient norm:      2e+01  
[10:32:54] < Debug >     ASC_MI:  -1.710806
[10:32:54] < Debug >     ASC_OC: -0.3115394
[10:32:54] < Debug >     ASC_RY:  0.9198199
[10:32:54] < Debug >     ASC_SC:  -1.211345
[10:32:54] < Debug >     ASC_SG:   2.678986
[10:32:54] < Debug >     ASC_YK:   2.015162
[10:32:54] < Debug >     B_DIS

[10:32:54] < Debug >     ASC_MI:  -5.419198
[10:32:54] < Debug >     ASC_OC:   1.089055
[10:32:54] < Debug >     ASC_RY:   1.789629
[10:32:54] < Debug >     ASC_SC: -0.5576171
[10:32:54] < Debug >     ASC_SG:   3.691025
[10:32:54] < Debug >     ASC_YK:   2.911087
[10:32:54] < Debug >     B_DIST: -0.06101184
[10:32:54] < General >   Log likelihood (N=233):  -238.3169 Gradient norm:        0.4  
[10:32:54] < Debug >     ASC_MI:  -5.755993
[10:32:54] < Debug >     ASC_OC:   1.075501
[10:32:54] < Debug >     ASC_RY:   1.772337
[10:32:54] < Debug >     ASC_SC: -0.6106892
[10:32:54] < Debug >     ASC_SG:   3.672571
[10:32:54] < Debug >     ASC_YK:   2.885797
[10:32:54] < Debug >     B_DIST: -0.06072932
[10:32:54] < General >   Log likelihood (N=233):  -238.3091 Gradient norm:        0.4  
[10:32:54] < Debug >     ASC_MI:  -6.590795
[10:32:54] < Debug >     ASC_OC:   1.085789
[10:32:54] < Debug >     ASC_RY:   1.658135
[10:32:54] < Debug >     ASC_SC: -0.8289563
[10:32:54] < Debug >     ASC_S

[10:32:54] < Debug >     ASC_RY:   1.762592
[10:32:54] < Debug >     ASC_SC: -0.6843811
[10:32:54] < Debug >     ASC_SG:   3.668023
[10:32:54] < Debug >     ASC_YK:   2.880114
[10:32:54] < Debug >     B_DIST: -0.06060838
[10:32:54] < General >   Log likelihood (N=233):  -238.2983 Gradient norm:        0.2  
[10:32:54] < Debug >     ASC_MI:   -15.0543
[10:32:54] < Debug >     ASC_OC:   1.070277
[10:32:54] < Debug >     ASC_RY:   1.762187
[10:32:54] < Debug >     ASC_SC: -0.6852318
[10:32:54] < Debug >     ASC_SG:   3.667527
[10:32:54] < Debug >     ASC_YK:   2.879501
[10:32:54] < Debug >     B_DIST: -0.06057436
[10:32:54] < General >   Log likelihood (N=233):  -238.2983 Gradient norm:       0.02  
[10:32:54] < Debug >     ASC_MI:  -15.59709
[10:32:54] < Debug >     ASC_OC:   1.070352
[10:32:54] < Debug >     ASC_RY:   1.762273
[10:32:54] < Debug >     ASC_SC: -0.6852449
[10:32:54] < Debug >     ASC_SG:    3.66761
[10:32:54] < Debug >     ASC_YK:    2.87955
[10:32:54] < Debug >     B_DIS

[10:32:54] < General >   Results saved in file SMTO_Segmented_Output/SMTO_Campus_Choice_Segment_12.html
[10:32:54] < General >   Results saved in file SMTO_Segmented_Output/SMTO_Campus_Choice_Segment_12.pickle
[10:32:54] < General >   Log file created: SMTO_Segmented_Output/SMTO_Campus_Choice_Segment_12.log
ASC_MI    =	-23.6
ASC_OC    =	1.07
ASC_RY    =	1.76
ASC_SC    =	-0.685
ASC_SG    =	3.67
ASC_YK    =	2.88
B_DIST    =	-0.0606
            Value       Std err    t-test       p-value  Rob. Std err  \
ASC_MI -23.648202  84963.240995 -0.000278  9.997779e-01      0.579407   
ASC_OC   1.070242      0.653991  1.636478  1.017396e-01      0.650781   
ASC_RY   1.762173      0.618126  2.850832  4.360498e-03      0.617966   
ASC_SC  -0.685325      1.156813 -0.592425  5.535660e-01      1.156292   
ASC_SG   3.667503      0.586227  6.256116  3.946821e-10      0.585351   
ASC_YK   2.879451      0.597897  4.815965  1.464900e-06      0.597874   
B_DIST  -0.060570      0.016723 -3.621862  2.924898e-04

In [9]:
pandasResults

,Value,Std err,t-test,p-value,Rob. Std err,Rob. t-test,Rob. p-value
ASC_MI,-23.648202,84963.240995,-0.000278,9.997779e-01,0.579407,-40.814486,0.000000e+00
ASC_OC,1.070242,0.653991,1.636478,1.017396e-01,0.650781,1.644549,1.000629e-01
ASC_RY,1.762173,0.618126,2.850832,4.360498e-03,0.617966,2.851570,4.350388e-03
ASC_SC,-0.685325,1.156813,-0.592425,5.535660e-01,1.156292,-0.592692,5.533872e-01
ASC_SG,3.667503,0.586227,6.256116,3.946821e-10,0.585351,6.265474,3.716929e-10
ASC_YK,2.879451,0.597897,4.815965,1.464900e-06,0.597874,4.816153,1.463526e-06
B_DIST,-0.060570,0.016723,-3.621862,2.924898e-04,0.017145,-3.532831,4.111349e-04
